In [2]:
##### Cleans EU capital stock and labor data
# convert to totals and reformat

import os
import pandas as pd

In [3]:
##### Load data

# Get the current working directory
cd = os.path.dirname(os.getcwd())

# Import data
EU_data = pd.read_csv(f"{cd}/Data/Raw/EU_FADN/capital_labor_01152026.csv")

# EU_geo = pd.read_csv(f"{cd}/Data/Correspondence_tables/XXX.csv")

# Set save path
save_path_capital = f"{cd}/Data/Clean/Capital_stock/EU_FADN_capital_stock.csv"
save_path_labor = f"{cd}/Data/Clean/Labor/EU_FADN_labor.csv"